# Project setup

In [ ]:
import os
import sys
import yaml
import pandas as pd
import tools.objects as objs
import tools.helpers as hlp
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', 500)

In [ ]:
# Optional: list all configurations available to load with the 'config_path' argument below
hlp.list_persistent_configs()

In [ ]:
# Initialize project (add the data processing and analysis hashes if resuming a previous project)
project = objs.Project(data_processing_hash=None, analysis_hash=None, overwrite=False)

# Dataset initiation

In [ ]:
# Create dataset objects and gather raw metadata and data (already pre-obtained from JGI sources)
tx_dataset = objs.TX(project)
mx_dataset = objs.MX(project, last=True)

In [ ]:
mx_dataset.raw_metadata.head()

In [ ]:
tx_dataset.raw_metadata.head()

# Analysis initiation

In [ ]:
# Create analysis object (collection of datasets and methods for performing integration)
analysis = objs.Analysis(project, datasets=[tx_dataset, mx_dataset])

# Data processing

In [ ]:
# Link analysis datasets by finding corresponding sample metadata fields
analysis.link_metadata()

In [ ]:
# Link analysis datasets matrices by using linked metadata
analysis.link_data()

In [ ]:
# Filter out rare features from analysis datasets based on minimum observed value or proportion of missing values across samples
analysis.filter_all_datasets()

In [ ]:
# Filter out features from analysis datasets that were not impacted by experimentation based on low variance across samples
analysis.devariance_all_datasets()

In [ ]:
# Scale features in all analysis datasets to a shared, normalized distribution
analysis.scale_all_datasets()

In [ ]:
# Filter out features from analysis datasets based on low within-replicate reproducibility
analysis.replicability_test_all_datasets()

In [ ]:
# View non-normalized data distributions
hlp.plot_simple_histogram(mx_dataset.linked_data, plot_title="Metabolite Peak Heights", output_dir=analysis.output_dir)
hlp.plot_simple_histogram(tx_dataset.linked_data, plot_title="Transcript Counts", output_dir=analysis.output_dir)

In [ ]:
# View normalized data distributions
analysis.plot_dataset_distributions()

In [ ]:
# Check dimension reduction plots before and after data normalization steps
analysis.plot_pca_all_datasets()

# Integration analysis

In [ ]:
# Integrate metadata tables by overlapping samples
analysis.integrate_metadata()

In [ ]:
# View metadata after integration
analysis.integrated_metadata.head()

In [ ]:
# Integrate data matrices by overlapping samples
analysis.integrate_data()

In [ ]:
# View data after integration
analysis.integrated_data.head()

In [ ]:
analysis.integrated_data.shape

In [ ]:
# Annotate the integrated features with pre-generated feature annotation tables
analysis.annotate_integrated_features()

In [ ]:
# Subset features using statistical tests
analysis.perform_feature_selection()

In [ ]:
# View PCA after integration to examine clustering pattern of combined data types
hlp.plot_simple_pca(analysis.integrated_data_selected, analysis.integrated_metadata, ["timepoint", "salinity", "light"], output_dir=analysis.output_dir)

In [ ]:
# Calculate correlations with custom parameters
analysis.calculate_correlated_features()

In [ ]:
# Plot correlation network
analysis.plot_correlation_network()

# Exploration of Results

In [ ]:
# Assess enrichment of an annotation layer in the extracted submodules
analysis.perform_functional_enrichment()

In [ ]:
# View average node abundance of a given submodule
analysis.plot_submodule_avg_abundance(submodule_name='submodule_1', metadata_cat='group', save_plot=True)

In [ ]:
# Sync all results tables to database for query actions
analysis.register_all_existing_data()

In [ ]:
# # Examples of natural language queries for data exploration
submod_request_ex = analysis.query("Show edges the network's submodule 1 where both the nodes does not have an 'Unassigned' annotation")
display(submod_request_ex.head())

In [ ]:
# View abundance patterns of individual features
#analysis.plot_individual_feature(feature_id='', metadata_cat='group', save_plot=True)

# Network Comparison 

In [ ]:
# Assess the loss and gain of information with dataset integration
analysis.run_full_network_analyzer()

# Save configuration and notebook

In [ ]:
# Save persistent configuration and notebook files
project.save_persistent_config_and_notebook()